In [1]:
import tensorflow as tf
from tensorflow.keras import regularizers, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

2025-01-27 09:39:31.785423: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-27 09:39:32.082764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737970772.184951    5174 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737970772.214421    5174 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-27 09:39:32.446754: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [3]:
# %% Load Data
data = pd.read_csv("balanced_soldier_data.csv")

In [4]:
print(data.info())  # Check for NaNs and types
print(data.describe())  # Feature distributions

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5100 entries, 0 to 5099
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Temperature       5100 non-null   float64
 1   Moisture          5100 non-null   float64
 2   Water_Content     5100 non-null   float64
 3   SpO2              5100 non-null   float64
 4   Fatigue           5100 non-null   float64
 5   Drowsiness        5100 non-null   float64
 6   Stress            5100 non-null   float64
 7   Heart_Rate        5100 non-null   float64
 8   Respiration_Rate  5100 non-null   float64
 9   Systolic_BP       5100 non-null   int64  
 10  Diastolic_BP      5100 non-null   int64  
 11  Efficiency        5100 non-null   float64
dtypes: float64(10), int64(2)
memory usage: 478.3 KB
None
       Temperature     Moisture  Water_Content         SpO2      Fatigue  \
count  5100.000000  5100.000000    5100.000000  5100.000000  5100.000000   
mean     36.824245    49.

In [5]:
X = data.drop(columns=["Efficiency"])
y = data["Efficiency"]

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [7]:
# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [8]:
# %% Build Model
def build_model(input_shape):
    model = Sequential(
        [
            layers.Input(shape=(input_shape,)),
            layers.Dense(
                128, activation="relu", kernel_regularizer=regularizers.l2(0.01)
            ),
            layers.BatchNormalization(),
            layers.Dropout(0.2),
            layers.Dense(
                256, activation="relu", kernel_regularizer=regularizers.l2(0.01)
            ),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            layers.Dense(128, activation="relu"),
            layers.Dense(64, activation="relu"),
            layers.Dense(32, activation="relu"),
            layers.Dense(1, activation="linear"),
        ]
    )
    return model

In [9]:
model = build_model(X_train.shape[1])

I0000 00:00:1737970777.971299    5174 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,361 (310.00 KB)

 Trainable params: 78,593 (307.00 KB)

 Non-trainable params: 768 (3.00 KB)

In [10]:
# Compile model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="mean_squared_error", metrics=["mae"])

In [11]:
# %% Train Model (Uncomment this to train)
# history = model.fit(
#     X_train,
#     y_train,
#     epochs=100,
#     batch_size=32,
#     validation_data=(X_test, y_test),
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(
#             monitor="val_loss", patience=10, restore_best_weights=True
#         )
#     ],
# )

In [12]:
# %% Evaluate Model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}, Test MAE: {test_mae:.4f}")

I0000 00:00:1737970779.106167    5389 service.cc:148] XLA service 0x7f94780083e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1737970779.106581    5389 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-01-27 09:39:39.128089: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1737970779.183813    5389 cuda_dnn.cc:529] Loaded cuDNN version 90300


31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.3822 - mae: 1.8530

I0000 00:00:1737970780.109887    5389 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 6.3796 - mae: 1.8524
Test Loss: 6.3405, Test MAE: 1.8421


In [13]:
# # %% Anomaly Detection
# def predict_anomaly(model, scaler, sample_input):
#     efficiency_score = model.predict(sample_input).flatten()[0]

#     print(f"Predicted Efficiency Score: {efficiency_score:.2f}")
#     if efficiency_score <= 0.3:
#         print("🚨 ALERT: Soldier in danger! Triggering rescue signal! 🚨")
#     else:
#         print("✅ Soldier efficiency is normal.")

In [14]:
# %% Save Model
# model.save("TrainedModel/SHMS_Efficiency_Model.keras")

In [15]:
# import joblib

# # Save the fitted scaler
# joblib.dump(scaler, "TrainedModel/scaler.pkl")
# print("Scaler saved successfully.")

In [16]:
msg = {
    "Timestamp": [
        "2025-01-26 15:36:19.880302",
        "2025-01-26 15:36:19.880686",
        "2025-01-26 15:36:19.880791",
        "2025-01-26 15:36:19.880872",
        "2025-01-26 15:36:19.880947",
        "2025-01-26 15:36:19.881030",
        "2025-01-26 15:36:19.881106",
        "2025-01-26 15:36:19.881179",
        "2025-01-26 15:36:19.881288",
        "2025-01-26 15:36:19.881381",
    ],
    "Temperature": [
        37.448019359758725,
        36.55113168092435,
        39.960331577182224,
        35.27576724919126,
        36.431665945230215,
        39.83317895095812,
        36.7614112059241,
        38.73903536889354,
        35.1212401581978,
        35.911808779094734,
    ],
    "Moisture": [
        42.56420788806833,
        39.0938031314282,
        17.688072988907177,
        57.61316107794047,
        40.45428780319771,
        11.469823784936391,
        33.150169544313094,
        20.314070455983696,
        65.03201425787528,
        50.09846667385073,
    ],
    "Water_Content": [
        57.09519357068937,
        55.0589017209392,
        23.454573754654337,
        61.93531588362247,
        58.27967325743154,
        24.93848251004362,
        49.35336596580625,
        36.85348865259063,
        76.32503367007183,
        61.95523086380585,
    ],
    "SpO2": [
        90.83170716027566,
        94.03622101913521,
        82.81685532148833,
        99.5673519759851,
        90.52837718064711,
        80.36215954970702,
        91.76107125147107,
        82.57934264809823,
        95.53706058963472,
        99.89370379497696,
    ],
    "Fatigue": [
        45.0518749069516,
        63.29676145438248,
        84.72875285481501,
        19.237833558594502,
        52.43043321802156,
        94.69959782044042,
        53.03885894986844,
        81.45319882475768,
        31.02373622644415,
        15.341721781946449,
    ],
    "Drowsiness": [
        47.387886875025146,
        34.84353162511356,
        89.8256501414069,
        26.18278236445019,
        46.12044436003424,
        81.29019836534223,
        54.61491499955926,
        89.71582996078122,
        21.33491160150549,
        12.89016438003876,
    ],
    "Stress": [
        32.45078097805382,
        45.65658629139096,
        98.9954187122699,
        13.934684709270357,
        45.48335816304045,
        86.4486021517516,
        34.98242144605149,
        77.78322491281463,
        27.459439032279775,
        27.47415175644288,
    ],
    "Heart_Rate": [
        98.67888124949874,
        90.65445102827275,
        115.04876178835602,
        60.84586650748115,
        91.44316814717094,
        122.80140854933657,
        94.1981196914343,
        115.62081753302822,
        79.29264396251105,
        75.5036735308056,
    ],
    "Respiration_Rate": [
        24.90624758267431,
        22.905683005007393,
        34.59183284133144,
        16.81292992972374,
        21.290915564703873,
        27.062843902030057,
        22.810069248529683,
        33.13968625079214,
        14.638286021824017,
        12.816054423725348,
    ],
    "Systolic_BP": [122, 128, 137, 119, 129, 131, 125, 131, 114, 111],
    "Diastolic_BP": [75, 84, 85, 76, 75, 89, 84, 85, 75, 70],
}

In [17]:
Results = []
Temperature = msg["Temperature"]
Moisture = msg["Moisture"]
Water_Content = msg["Water_Content"]
SpO2 = msg["SpO2"]
Fatigue = msg["Fatigue"]
Drowsiness = msg["Drowsiness"]
Stress = msg["Stress"]
Heart_Rate = msg["Heart_Rate"]
Respiration_Rate = msg["Respiration_Rate"]
Systolic_BP = msg["Systolic_BP"]
Diastolic_BP = msg["Diastolic_BP"]
if model:
    for x in range(0, len(Temperature)):
        input_data = np.array([
            Temperature[x],
            Moisture[x],
            Water_Content[x],
            SpO2[x],
            Fatigue[x],
            Drowsiness[x],
            Stress[x],
            Heart_Rate[x],
            Respiration_Rate[x],
            Systolic_BP[x],
            Diastolic_BP[x],
        ]).reshape(1,-1)

        # Scale input data using pre-fitted scaler

        # Perform prediction
        efficiency_score = model.predict(input_data).flatten()[0]
        Results.append(efficiency_score)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [18]:
Results

[2.8855686,
 3.529185,
 3.2032003,
 1.5475178,
 2.901136,
 3.8419626,
 3.1833763,
 3.693666,
 1.5080633,
 1.8627253]